## Домашнее задание

Откуда берутся датасеты? Практический проект по сбору данных и работе с текстами

Цель:

В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-код, собрать себе свой собственный датасет и натренировать на нём модель.

## Часть 1. Парсинг.


По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).


Идеальный датасет должен иметь текстовое описание некоторого объекта и некоторую целевую переменную, соответствующую этому объекту. Например:

- Сайт новостей: текстовое описание - сама новость, целевая переменная - количество просмотров новости (можно поделить на число дней с момента даты публикации, чтобы получить “среднее число просмотров в день”).
- Сайт с товарами/книгами/фильмами: текстовое описание товара/книги/фильма + средний рейтинг в качестве целевой переменной.
- Блоги - тексты заметок + число просмотров.
- И любые другие ваши идеи, которые подходят под такой формат.

Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.
Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла) 

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   
import pandas as pd  
import time          

import warnings
warnings.filterwarnings("ignore")

from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm import tqdm

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием BeautifulSoup — скорее всего, не то, что нам нужно. Это третья версия (Beautiful Soup 3), а мы будем использовать четвертую. Нужно будет установить пакет beautifulsoup4. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — bs4, а импортировать функцию под названием BeautifulSoup. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет xml.

Собирать данные будем на новостном сайте Nice Matin - утренняя Ницца )

In [2]:
top_page = 'https://www.nicematin.com/'

In [3]:
UserAgent().chrome

'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36'

In [4]:
top_response = requests.get(top_page, headers={'User-Agent': UserAgent().chrome})
top_response

<Response [200]>

In [5]:
top_html = top_response.content

In [6]:
len(top_html)

273989

In [7]:
top_soup = BeautifulSoup(top_html, 'html.parser')

In [8]:
print(top_soup.prettify())

<!DOCTYPE html>
<html class="nm" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="icon" type="image/png"/>
  <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon"/>
  <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="57x57">
   <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="60x60">
    <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="72x72"/>
    <link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="76x76"/>
    <lin

In [9]:
top_soup.html.head.title

<title>Actualités et infos en direct - Nice-Matin</title>

Напишем функцию для получения списка ссылок на странице

In [10]:
def get_links_list (page, tag, class_to_find):
    """
    Parameters:
    page: link to an html-page to find the links on;
    tag: <tag> to find;
    class_to_find: <class> to find; 
    
    Returns: list of links
    """
    response = requests.get(page, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    obj_list = soup.find_all(tag, attrs = {'class':class_to_find})
    links_list = [obj_item.attrs['href'] for obj_item in obj_list if obj_item.attrs['href'][0:25] == 'https://www.nicematin.com']
    return links_list

Соберем список тематических ссылок на первой странице Nice matin

In [11]:
theme_links_list = get_links_list(top_page, 'a', 'header-link-secondary')

In [12]:
theme_links_list

['https://www.nicematin.com/sujet/faits-de-societe',
 'https://www.nicematin.com/sujet/faits-divers',
 'https://www.nicematin.com/sujet/economie',
 'https://www.nicematin.com/sujet/politique',
 'https://www.nicematin.com/sujet/sport',
 'https://www.nicematin.com/sujet/sante',
 'https://www.nicematin.com/sujet/ogcnice',
 'https://www.nicematin.com/sujet/opinion',
 'https://www.nicematin.com/sujet/culture',
 'https://www.nicematin.com/sujet/jeux-olympiques',
 'https://www.nicematin.com/ville/nice',
 'https://www.nicematin.com/commune/saint-laurent-du-var',
 'https://www.nicematin.com/ville/cagnes',
 'https://www.nicematin.com/ville/cannes',
 'https://www.nicematin.com/ville/antibes',
 'https://www.nicematin.com/ville/grasse',
 'https://www.nicematin.com/ville/menton',
 'https://www.nicematin.com/ville/monaco',
 'https://www.nicematin.com/ville/vallees']

In [13]:
theme_links_list[0]

'https://www.nicematin.com/sujet/faits-de-societe'

Пройдем по всем тематическим ссылкам и соберем итоговый список ссылок на новостные статьи

In [14]:
allnews_links_list = []

In [15]:
for link in tqdm(theme_links_list):
    allnews_links_list = allnews_links_list + get_links_list(link, 'a', 'title-link')
    time.sleep(0.3)

100%|███████████████████████████████████████████| 19/19 [00:15<00:00,  1.20it/s]


In [16]:
len(allnews_links_list)

294

In [17]:
allnews_links_list

['https://www.nicematin.com/education/on-fait-ce-stage-par-depit-clairement-926616',
 'https://www.nicematin.com/faits-de-societe/-face-a-leur-usage-des-ecrans-les-jeunes-appellent-a-l-aide-une-commission-d-experts-propose-de-limiter-l-usage-du-smartphone-en-fonction-de-l-age-926348',
 'https://www.nicematin.com/faits-de-societe/face-aux-deserts-medicaux-ce-bus-prodigue-des-soins-dentaires-gratuits-en-milieu-rural-926111',
 'https://www.nicematin.com/faits-de-societe/giulia-sarkozy-ambassadrice-d-une-marque-varoise-926402',
 'https://www.nicematin.com/faits-de-societe/au-texas-les-anti-ivg-veulent-aller-plus-loin-et-empecher-les-femmes-daller-avorter-ailleurs-926448',
 'https://www.nicematin.com/environnement/avec-c-monspotfr-on-fait-du-sport-en-respectant-la-nature-926108',
 'https://www.nicematin.com/faits-de-societe/-mon-vol-annule-pour-la-3e-fois-un-hyerois-bloque-en-nouvelle-caledonie-raconte-926363',
 'https://www.nicematin.com/cinema/festival-de-cannes-mediapart-publie-une-nouve

Посмотрим на первую ссылку в списке

In [18]:
allnews_links_list[0]

'https://www.nicematin.com/education/on-fait-ce-stage-par-depit-clairement-926616'

попробуем получить детальную информацию по новости

In [19]:
response = requests.get(allnews_links_list[0], headers={'User-Agent': UserAgent().chrome})

In [20]:
response

<Response [200]>

In [21]:
html = response.content

In [22]:
soup = BeautifulSoup(html, 'html.parser')

In [23]:
soup

<!DOCTYPE html>

<html class="nm" lang="fr">
<head>
<meta charset="utf-8"/>
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="icon" type="image/png"/>
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="shortcut icon" type="image/x-icon"/>
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon"/>
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="57x57">
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="60x60">
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="https://cdn.assets01.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="https://cdn.asset

In [24]:
title = soup.find('h1', attrs = {'class':"article-title"}).text
title

'"On le fait par dépit, clairement": la galère des lycéens de seconde à Antibes pour trouver un stage de fin d\'année'

In [25]:
excerpt = soup.find('h2', attrs = {'class':"article-excerpt"}).text.strip('\r\n ')
excerpt

'Pour la première fois, les élèves de seconde devront faire un stage. Certains essuient des refus et se rabattent sur des secteurs qui ne sont pas en lien avec leurs études.'

In [26]:
author = soup.find('span', attrs = {'class':"author"})
if author: author = author.text
author

'Nathan Beaufils'

In [27]:
published = soup.find('span', attrs = {'class':"published-at"})
if published: published = published.text
published

'Publié le 10/06/2024 à 08:45, mis à jour le 10/06/2024 à 08:45'

In [28]:
city = soup.find('a', attrs = {'class':"blue city"})
if city: city = city.text
city

'Antibes'

In [29]:
full_text = soup.find('section', attrs = {'class':"article-block", 'id':'full-text'})
if full_text: full_text = full_text.text.strip('\ncommentaires ')
full_text

'Devant le lycée Jacques-Audiberti, les élèves sortent les uns après les autres. C’est la récréation, l’une des dernières avant la fin de l’année.\nPour les classes de seconde générale et technologique, la fin de l’année ne se déroulera pas au lycée, mais en milieu professionnel, pour un stage de deux semaines, du 17 au 28 juin. Une nouvelle mesure prise à la rentrée par Gabriel Attal.\n"Essaie de faire ce choix en lien avec ton projet d’études", s’adressait aux lycéens l’ancien ministre de l’Éducation nationale sur son compte TikTok en novembre dernier.\nDes élèves sans stage \nPour ces jeunes étudiants, trouver un stage qui leur plaît n’est pas une mince affaire. "J’ai demandé dans des magasins, mais ils ne prennent pas à notre âge (15 ans, Ndlr). J’ai donc fait appel à ma mère pour travailler avec elle dans une entreprise de nettoyage", regrette Alexia.\nSon amie Manon a aussi essuyé plusieurs refus de la part de boutiques de produits de beauté.\xa0"On fait ce stage par dépit, clair

Напишем функцию, возвращающую информацию из новости (title, author, published, region, excerpt, full_text) по url

In [30]:
def getNewsData (nlink):
    
    response = requests.get(nlink, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h1', attrs = {'class':"article-title"})
    if title: title = title.text

    author = soup.find('span', attrs = {'class':"author"})
    if author: author = author.text
    
    published = soup.find('span', attrs = {'class':"published-at"})
    if published: published = published.text

    city = soup.find('a', attrs = {'class':"blue city"})
    if city: city = city.text
    
    excerpt = soup.find('h2', attrs = {'class':"article-excerpt"})
    if excerpt: excerpt = excerpt.text.strip('\r\n ')

    full_text = soup.find('section', attrs = {'class':"article-block", 'id':'full-text'})
    if full_text: full_text = full_text.text.strip('\ncommentaires ')

    data_row = {'Title':title, 'Author':author, 'Published':published, 'Region':city,
                'Excerpt':excerpt, 'Full_text':full_text}   
    
    return data_row

In [31]:
allnews_links_list[13]

'https://www.nicematin.com/faits-de-societe/et-si-cette-drole-de-maison-en-forme-de-a-etait-lavenir-de-la-construction-925416'

In [32]:
data_row = getNewsData(allnews_links_list[13])

In [33]:
data_row

{'Title': "Et si cette drôle de maison en forme de A était l'avenir de la construction?",
 'Author': 'Emma Gouaille - Sud Ouest',
 'Published': 'Publié le 07/06/2024 à 18:45, mis à jour le 07/06/2024 à 18:45',
 'Region': 'France',
 'Excerpt': "Face au changement climatique, comment trouver des solutions concrètes pour habiter la Terre plus sobrement? A l'occasion de la Journée internationale de l’environnement, le groupe Nice-Matin s’associe à Sparknews et 51 autres titres de la presse quotidienne régionale pour, ensemble, explorer les initiatives qui repensent déjà nos territoires. Halte en Dordogne, pour découvrir une construction atypique, qui souhaite répondre aux défis environnementaux et économiques de demain. Et d'aujourd'hui.",
 'Full_text': 'Une véritable communauté s’est créée autour de cet habitat atypique. Ils sont près de 50.000 membres sur le groupe Facebook "La maison en A". Sur le site officiel, la carte recense des dizaines de projets: ici, le permis de construire est 

Работает ! Осталось пройти в цикле по всем новостным ссылкам и сохранить полученные данные в pandas dataframe

In [34]:
df_news = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Region','Excerpt', 'Full_text'])

In [35]:
for news_link in tqdm(allnews_links_list):
    data_row = getNewsData(news_link)
    df_news = pd.concat([df_news, pd.DataFrame([data_row])], ignore_index=True)
    time.sleep(0.3)

100%|█████████████████████████████████████████| 294/294 [04:40<00:00,  1.05it/s]


In [36]:
df_news.shape

(294, 6)

In [37]:
df_news.tail()

,Title,Author,Published,Region,Excerpt,Full_text
289,Une brebis dévorée et deux autres blessées à S...,Mathilde Tranoy,"Publié le 08/06/2024 à 19:45, mis à jour le 08...",Vallées,Une brebis a été retrouvée morte et deux autre...,Macabre découverte ce samedi matin à Saint-Eti...
290,"Face au changement climatique, deux modes d'em...",Aurélie Selvi - aselvi@nicematin.fr,"Publié le 08/06/2024 à 09:45, mis à jour le 08...",Vence,"Face au changement climatique, comment trouver...","En France, de 2010 à 2020, une surface équival..."
291,Un nouveau rond-point inauguré dans l’arrière-...,Benoit Guglielmibguglielmi@nicematin.fr,"Publié le 08/06/2024 à 07:52, mis à jour le 08...",Vallées,Le giratoire Charles-Ginésy doit être inauguré...,Plus de sécurité: voilà ce qu’est censé apport...
292,"TAMA, 60 ans de travaux en famille",K.Wenger,"Publié le 07/06/2024 à 14:44, mis à jour le 07...",Nice,"L’entreprise de travaux publics cagnoise TAMA,...","Juin 1964. Antoine Renaudi, 28 ans, possède un..."
293,"400 élèves, un établissement à ""taille humaine...",E. G.,"Publié le 07/06/2024 à 10:53, mis à jour le 07...",Vallées,Le conseil départemental des Alpes-Maritimes a...,"Le collège Roger-Carlès de Contes, prévu pour ..."


### Вывод
Получилось собрать данные с новостного сайта Nice-Matin, но поскольку датасет получился небольшим и в нем не оказалось целевой переменной ), то для второй части использую датасет IMDB. И спасибо за задание - его выполнение доставило мне искреннее удовольствие, словно бродил не по сайту, а по улицам Ниццы...